# 库

In [5]:
import numpy as np
import mnist 
np.random.seed(114514)  #随机数种子

# 卷积核(滤波器)类

In [6]:
class Conv3x3:
    #3*3的卷积层
    
    def __init__(self, num_filters):
        
        self.num_filters = num_filters
        #num_filters即为滤波器的数量
        
        self.filters = np.random.uniform(-1, 1, (num_filters, 3, 3))
        #将滤波器权重初始化为均匀分布的值（矩阵中的每个元素大小都在-1到1之间）
        
    def iterate_regions(self, image):
        #设定迭代区域
        
        h, w = image.shape
        #h表示图像高度，w表示宽度
        
        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                # 生成图像的所有可能的 3x3 子区域
                
                yield im_region, i, j
                #将子区域的信息存入迭代器
 
    def forward(self, input):
        #向前传播函数
        
        h, w = input.shape
        #mnist数据集的图片大小为28*28,因此input也是28*28
        
        output = np.zeros((h - 2, w - 2, self.num_filters))
        #output为26*26*num_filters
 
        for im_region, i, j in self.iterate_regions(input):
        
            # 卷积运算，点乘再相加，ouput[i, j] 为向量，8 层
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
            
        # 最后将输出数据返回，便于下一层的输入使用
        return output

# 池化层类

In [10]:
class MaxPool2:
    # 2*2的池化层
    
    def iterate_regions(self,image):
        #此时的image为26*26*num_filters的三维数组
        
        h,w,_=image.shape
        # _表示不获取它的深度，即有几层
        
        new_h = h//2 #类似C语言的 h/2
        new_w = w//2
        
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i*2):(i*2+2),(j*2):(j*2+2)]
                yield im_region,i,j
        #上面是和卷积核类同样的切片操作
        
    def forward(self,input):
        
        h,w,num_filters = input.shape
        #获取image的三围
        
        output = np.zeros((h//2,w//2,num_filters))
        
        for im_region,i,j in self.iterate_regions(input):
            output[i,j] = np.amax(im_region,axis=(0,1))
            #计算之前切片操作的那个2*2矩阵里的元素的最大值
            
        return output 

# 全连接层类

In [11]:
class Softmax:
    # 用Softmax函数激活的全连接层
 
    def __init__(self, input_len, nodes):
        # input_len为输入层的节点个数(池化层处理之后的)
        # nodes为输出层的节点个数，因为需要识别数字0到9，因此结点个数为10
        
        self.weights = np.random.randn(input_len, nodes) / input_len
        # 构建权重矩阵，初始化随机数，不能太大
        #除以input_len以减少初始值的方差
        
        self.biases = np.zeros(nodes)
        #偏置向量用于在计算每个类别的得分时添加一个偏移量
 
    def forward(self , input):

        input = input.flatten()
        #三维数组降成一维数组，用于构建全连接层
        #例：[[1, 2, 3]       拉平后：[1, 2, 3, 4, 5, 6]    
        #     [4, 5, 6]]
 
        input_len, nodes = self.weights.shape
 
        totals = np.dot(input, self.weights) + self.biases
        #将输入数据 input 与权重矩阵 self.weights 进行点积操作，然后将得到的每个类别的得分加上相应的偏置，得到最终的 totals 数组。这个 totals 数组代表了每个类别的得分
        
        exp = np.exp(totals)
        
        return exp / np.sum(exp, axis=0)
        #softmax函数计算

In [17]:
test_images = mnist.test_images()[:2000]
test_labels = mnist.test_labels()[:2000]
 
conv = Conv3x3(8)                                    
pool = MaxPool2()                                    
softmax = Softmax(13 * 13 * 8, 10) 
 
def forward(image, label):
   
    out = conv.forward((image / 255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0
 
    return out, loss, acc
 
print('MNIST数据集上的CNN初始化成功')
 
loss = 0
num_correct = 0

for i, (im, label) in enumerate(zip(test_images, test_labels)):

    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc
 
    if i % 100 == 99:
        print(
            '[Step %d] 过去一百张图片: 平均损失值 %.3f | 准确率: %d%%' %
            (i + 1, loss / 100, num_correct)
        )
        loss = 0
        num_correct = 0

MNIST数据集上的CNN初始化成功
[Step 100] 过去一百张图片: 平均损失值 2.307 | 准确率: 5%
[Step 200] 过去一百张图片: 平均损失值 2.304 | 准确率: 9%
[Step 300] 过去一百张图片: 平均损失值 2.305 | 准确率: 6%
[Step 400] 过去一百张图片: 平均损失值 2.305 | 准确率: 13%
[Step 500] 过去一百张图片: 平均损失值 2.304 | 准确率: 10%
[Step 600] 过去一百张图片: 平均损失值 2.304 | 准确率: 7%
[Step 700] 过去一百张图片: 平均损失值 2.305 | 准确率: 6%
[Step 800] 过去一百张图片: 平均损失值 2.306 | 准确率: 12%
[Step 900] 过去一百张图片: 平均损失值 2.306 | 准确率: 9%
[Step 1000] 过去一百张图片: 平均损失值 2.306 | 准确率: 8%
[Step 1100] 过去一百张图片: 平均损失值 2.305 | 准确率: 7%
[Step 1200] 过去一百张图片: 平均损失值 2.303 | 准确率: 10%
[Step 1300] 过去一百张图片: 平均损失值 2.304 | 准确率: 10%
[Step 1400] 过去一百张图片: 平均损失值 2.306 | 准确率: 7%
[Step 1500] 过去一百张图片: 平均损失值 2.304 | 准确率: 5%
[Step 1600] 过去一百张图片: 平均损失值 2.305 | 准确率: 6%
[Step 1700] 过去一百张图片: 平均损失值 2.303 | 准确率: 11%
[Step 1800] 过去一百张图片: 平均损失值 2.306 | 准确率: 8%
[Step 1900] 过去一百张图片: 平均损失值 2.305 | 准确率: 10%
[Step 2000] 过去一百张图片: 平均损失值 2.304 | 准确率: 7%
